In [2]:
# Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from dotenv import load_dotenv, find_dotenv
import os

# Загрузка переменных окружения
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    print("Файл .env не найден")

# Проверка наличия переменной окружения API_URL
api_url = os.getenv("API_URL")
if not api_url:
    raise ValueError("Переменная окружения API_URL не найдена. Проверьте файл .env")

# Функции для запроса данных по API
def fetch_visits(api_url, start_date, end_date):
    response = requests.get(f"{api_url}/visits", params={'begin': start_date, 'end': end_date})
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

def fetch_registrations(api_url, start_date, end_date):
    response = requests.get(f"{api_url}/registrations", params={'begin': start_date, 'end': end_date})
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Запрос данных
start_date = "2023-03-01"
end_date = "2023-09-01"

visits_data = fetch_visits(api_url, start_date, end_date)
registrations_data = fetch_registrations(api_url, start_date, end_date)

# Преобразование данных в DataFrame
visits_df = pd.DataFrame(visits_data)
registrations_df = pd.DataFrame(registrations_data)

# Удаление визитов ботов
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]

# Преобразование колонок datetime в тип datetime
visits_df['datetime'] = pd.to_datetime(visits_df['datetime'])
registrations_df['datetime'] = pd.to_datetime(registrations_df['datetime'])

# Добавление колонки date_group, которая содержит только дату (без времени)
visits_df['date_group'] = visits_df['datetime'].dt.date
registrations_df['date_group'] = registrations_df['datetime'].dt.date

# Группировка данных визитов
visits_agg = visits_df.groupby(['date_group', 'platform']).size().reset_index(name='visits')

# Группировка данных регистраций
registrations_agg = registrations_df.groupby(['date_group', 'platform']).size().reset_index(name='registrations')

# Объединение данных визитов и регистраций
merged_df = pd.merge(visits_agg, registrations_agg, on=['date_group', 'platform'], how='left')
merged_df['registrations'] = merged_df['registrations'].fillna(0)  # Заполнение NaN значений в регистрациях нулями

# Расчет конверсии
merged_df['conversion'] = (merged_df['registrations'] / merged_df['visits']) * 100

# Сортировка данных по дате
merged_df = merged_df.sort_values(by='date_group')

# Сохранение данных в формате JSON
merged_df.to_json('conversion.json', orient='records', date_format='iso')

# Вывод результата
print(merged_df)


     date_group platform  visits  registrations  conversion
0    2023-03-01  android      75             61   81.333333
1    2023-03-01      ios      22             18   81.818182
2    2023-03-01      web     844              8    0.947867
3    2023-03-02  android      67             59   88.059701
4    2023-03-02      ios      31             24   77.419355
..          ...      ...     ...            ...         ...
546  2023-08-30  android      35             27   77.142857
548  2023-08-30      web    1357             34    2.505527
550  2023-08-31      ios      50             36   72.000000
549  2023-08-31  android      57             42   73.684211
551  2023-08-31      web     722             46    6.371191

[552 rows x 5 columns]
